# Construir com os Modelos da Família Meta

## Introdução

Esta lição irá abordar:

- Explorar os dois principais modelos da família Meta - Llama 3.1 e Llama 3.2
- Compreender os casos de uso e cenários para cada modelo
- Exemplo de código para mostrar as características únicas de cada modelo

## A Família de Modelos Meta

Nesta lição, vamos explorar 2 modelos da família Meta ou "Rebanho Llama" - Llama 3.1 e Llama 3.2

Estes modelos existem em diferentes variantes e estão disponíveis no marketplace de Modelos do Github. Aqui tens mais detalhes sobre como usar os Github Models para [prototipar com modelos de IA](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).

Variantes dos Modelos:
- Llama 3.1 - 70B Instruct
- Llama 3.1 - 405B Instruct
- Llama 3.2 - 11B Vision Instruct
- Llama 3.2 - 90B Vision Instruct

*Nota: O Llama 3 também está disponível nos Github Models, mas não será abordado nesta lição*


## Llama 3.1

Com 405 mil milhões de parâmetros, o Llama 3.1 enquadra-se na categoria de LLMs open source.

Este modelo é uma atualização em relação ao Llama 3, oferecendo:

- Janela de contexto maior - 128k tokens vs 8k tokens
- Maior número máximo de tokens de saída - 4096 vs 2048
- Melhor suporte multilingue - devido ao aumento de tokens de treino

Estas melhorias permitem que o Llama 3.1 lide com casos de uso mais complexos ao desenvolver aplicações GenAI, incluindo:
- Chamada nativa de funções - capacidade de chamar ferramentas e funções externas fora do fluxo de trabalho do LLM
- Melhor desempenho em RAG - devido à janela de contexto superior
- Geração de dados sintéticos - capacidade de criar dados eficazes para tarefas como o fine-tuning


### Chamada Nativa de Funções

O Llama 3.1 foi ajustado para ser mais eficaz a fazer chamadas de funções ou ferramentas. Tem também duas ferramentas integradas que o modelo consegue identificar como necessárias, dependendo do pedido do utilizador. Estas ferramentas são:

- **Brave Search** – Pode ser usada para obter informações atualizadas, como o estado do tempo, através de uma pesquisa na web
- **Wolfram Alpha** – Pode ser usada para cálculos matemáticos mais complexos, evitando a necessidade de escrever as suas próprias funções.

Também pode criar as suas próprias ferramentas personalizadas que o LLM pode chamar.

No exemplo de código abaixo:

- Definimos as ferramentas disponíveis (brave_search, wolfram_alpha) no prompt do sistema.
- Enviamos um pedido do utilizador a perguntar pelo tempo numa determinada cidade.
- O LLM irá responder com uma chamada à ferramenta Brave Search, que terá este aspeto `<|python_tag|>brave_search.call(query="Stockholm weather")`

*Nota: Este exemplo apenas faz a chamada à ferramenta; se quiser obter os resultados, terá de criar uma conta gratuita na página da API do Brave e definir a função em si*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

Apesar de ser um LLM, uma das limitações do Llama 3.1 é a multimodalidade. Ou seja, a capacidade de utilizar diferentes tipos de input, como imagens, como prompts e fornecer respostas. Esta funcionalidade é uma das principais novidades do Llama 3.2. Estas funcionalidades incluem ainda:

- Multimodalidade - tem a capacidade de avaliar tanto prompts de texto como de imagem
- Variações de tamanho pequeno a médio (11B e 90B) - isto permite opções de implementação flexíveis,
- Variações apenas de texto (1B e 3B) - isto permite que o modelo seja implementado em dispositivos edge / móveis e oferece baixa latência

O suporte multimodal representa um grande avanço no mundo dos modelos open source. O exemplo de código abaixo utiliza tanto uma imagem como um prompt de texto para obter uma análise da imagem a partir do Llama 3.2 90B.

### Suporte Multimodal com o Llama 3.2


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## A aprendizagem não termina aqui, continua a tua Jornada

Depois de terminares esta lição, consulta a nossa [coleção de Aprendizagem de IA Generativa](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) para continuares a aprofundar os teus conhecimentos em IA Generativa!



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos pela precisão, tenha em atenção que traduções automáticas podem conter erros ou imprecisões. O documento original, na sua língua nativa, deve ser considerado a fonte autorizada. Para informações críticas, recomenda-se a tradução profissional por um humano. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas resultantes da utilização desta tradução.
